<a href="https://colab.research.google.com/github/jayrom/enterprise_challenge_reply/blob/main/sprint_3/src/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulação de dados de sensores industriais

### Importar módulos

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

### Configurações iniciais da simulação

In [2]:
NUM_DEVICES = 2
DAYS_TO_SIMULATE = 60
INTERVAL_MINUTES = 10
FAILURE_START_DAY = 30

### Parâmetros de dados normais (média e desvio padrão para ruído)

In [3]:
NORMAL_TEMP_MEAN = 45.0
NORMAL_TEMP_STD = 2.0
NORMAL_CURRENT_MEAN = 6.0
NORMAL_CURRENT_STD = 0.5
NORMAL_VIBRATION_MEAN = 0.25
NORMAL_VIBRATION_STD = 0.05

### Parâmetros de falha (taxa de aumento por dia)

In [4]:
FAULT_TEMP_INCREASE = 0.5  # Aumento de 0.5 °C por dia
FAULT_VIBRATION_INCREASE = 0.05  # Aumento de 0.05 g por dia
FAULT_CURRENT_INCREASE = 0.1      # Aumento de 0.1 A por dia

### Geração dos dados

In [5]:
all_data = []

# Loop para cada dispositivo
for i in range(1, NUM_DEVICES + 1):
    device_id = f'motor_{i}'
    start_time = datetime.now() - timedelta(days=DAYS_TO_SIMULATE)

    # Loop para cada ponto de dados
    for day in range(DAYS_TO_SIMULATE):
        for _ in range(0, 24 * 60, INTERVAL_MINUTES):
            current_time = start_time + timedelta(days=day, minutes=_)

            # Inicializa os dados com valores normais e ruído aleatório
            temp = np.random.normal(NORMAL_TEMP_MEAN, NORMAL_TEMP_STD)
            current = np.random.normal(NORMAL_CURRENT_MEAN, NORMAL_CURRENT_STD)
            vibration = np.random.normal(NORMAL_VIBRATION_MEAN, NORMAL_VIBRATION_STD)

            days_to_failure = -1  # -1 para indicar que não há falha
            failure_mode = 'normal'

            # Simula a falha progressiva para o motor_001
            if device_id == 'motor_1' and day >= FAILURE_START_DAY:
                days_since_start_of_failure = day - FAILURE_START_DAY

                # Aumenta os valores conforme a falha progride
                temp += days_since_start_of_failure * FAULT_TEMP_INCREASE
                vibration += days_since_start_of_failure * FAULT_VIBRATION_INCREASE
                current += days_since_start_of_failure * FAULT_CURRENT_INCREASE

                days_to_failure = DAYS_TO_SIMULATE - day
                failure_mode = 'failure_in_progress'

            temp = round(temp, 2)
            current = round(current, 2)
            vibration = round(vibration, 2)

            all_data.append([
                current_time,
                device_id,
                temp,
                current,
                vibration,
                days_to_failure,
                failure_mode
            ])

# Cria o DataFrame e salva no arquivo CSV
df = pd.DataFrame(all_data, columns=[
    'timestamp',
    'device_id',
    'temperature_c',
    'current_amps',
    'vibration_rms_g',
    'days_to_failure',
    'failure_mode'
])

output_filename = 'simulated_sensor_data.csv'
df.to_csv(output_filename, index=False)

print(f"Dados simulados salvos em '{output_filename}' com {len(df)} registros.")
print(df.head())
print(df.tail())

Dados simulados salvos em 'simulated_sensor_data.csv' com 17280 registros.
                   timestamp device_id  temperature_c  current_amps  \
0 2025-06-30 05:03:52.374174   motor_1          46.64          6.08   
1 2025-06-30 05:13:52.374174   motor_1          44.95          6.00   
2 2025-06-30 05:23:52.374174   motor_1          48.24          5.98   
3 2025-06-30 05:33:52.374174   motor_1          43.39          6.39   
4 2025-06-30 05:43:52.374174   motor_1          43.81          5.93   

   vibration_rms_g  days_to_failure failure_mode  
0             0.28               -1       normal  
1             0.16               -1       normal  
2             0.27               -1       normal  
3             0.20               -1       normal  
4             0.19               -1       normal  
                       timestamp device_id  temperature_c  current_amps  \
17275 2025-08-29 04:13:52.449093   motor_2          45.25          6.05   
17276 2025-08-29 04:23:52.449093   motor_2